In [2]:
import xlwings as xw

# Create a new Excel workbook
wb = xw.Book()

# Use the first sheet and rename it
sheet = wb.sheets[0]
sheet.name = "LiveData"

# Set column headers
sheet.range("A1:D1").value = ["Symbol", "LTP", "Bid Qty", "Ask Qty"]

# Save the workbook
wb.save("live_data.xlsx")
wb.close()

print("✅ Excel file 'live_data.xlsx' created successfully.")


✅ Excel file 'live_data.xlsx' created successfully.
